## Fantasy AI Assistant

This notebook will get you on your way to a fantasy football championship. Start by importing the Agent class we're going to use, setting the config list, and giving some system instructions. Note that `gpt-4o-mini` seems to work well. This model will be using a lot of tokens, running `gpt-4-turbo` with this notebook will ring up a large bill quickly, even with the token compression implemented below.

In [13]:
from autogen import ConversableAgent
import os 

config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]

assistant = ConversableAgent(
    name="Assistant",
    system_message="""You are a helpful AI assistant that is an expert in tool use and in fantasy football. It is long past October of 2023, you need to check the current date to search for current news.
    Think step-by-step about the steps needed to answer an input query, and use the tools available to you to get the information needed at each step.
    Before you recommend adding or dropping any player, be very thorough and check all tools available to you to get the most up to date information possible regarding that player.
    Return 'TERMINATE' when the task is done.""",
    llm_config={"config_list": config_list},
)

Creating the user proxy agent that acts as the stand in for you.

In [14]:
# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)


Now we start to create the tools that our assistant can use. There is potential for some change here, as these are written as tools rather than functions. When `autogen` hits their roadmap target of letting docker code execution agents take in user-defined functions, I will refactor this.

In [15]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from autogen import register_function

def google_it(query: str) -> str:
    "Search Google for keywords"
    #this is going to assume you have your environment variables set for GOOGLE_API and GOOGLE_CSE
    search = GoogleSearchAPIWrapper()

    google_search = Tool(
        name="google_search",
        description="Search Google for recent results.",
        func=search.run)
    
    return google_search({'query': query})

register_function(
    google_it,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="google_it",  # By default, the function name is used as the tool name.
    description="Searches google for recent results using the input as keywords",  # A description of the tool.
)


In [16]:
from langchain_community.tools.reddit_search.tool import RedditSearchRun
from langchain_community.utilities.reddit_search import RedditSearchAPIWrapper
from langchain_community.tools.reddit_search.tool import RedditSearchSchema

reddit_client_id = os.environ['REDDIT_CLIENT_ID']
reddit_client_secret = os.environ['REDDIT_CLIENT_SECRET']
reddit_user_agent = "fantasy-assistant"

def check_reddit(player: str, player_team_subreddit: str) -> str:
    "Check Reddit for any recent player news. Inputs are the player and the subreddit for that player's team"

    r_search = RedditSearchRun(
        api_wrapper=RedditSearchAPIWrapper(
            reddit_client_id=reddit_client_id,
            reddit_client_secret=reddit_client_secret,
            reddit_user_agent=reddit_user_agent,
        )
    )

    search_params = RedditSearchSchema(
        query=player, sort="new", time_filter="week", subreddit="fantasyfootball", limit="5"
        )

    result = r_search.run(tool_input=search_params.dict())

    try:
        search_params = RedditSearchSchema(
            query=player, sort="new", time_filter="week", subreddit=player_team_subreddit, limit="5"
            )
        extra_result = r_search.run(tool_input=search_params.dict())
        result += extra_result
    except:
        pass
    return result

register_function(
    check_reddit,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="check_reddit",  # By default, the function name is used as the tool name.
    description="Check Reddit for any recent player news. Inputs are the player and the subreddit for that player's team",  # A description of the tool.
)

Here you need to make sure you've grabbed your `swid` and `s2` from ESPN's website. It's fairly straightforward when you go to developer tools after logging in. You also need to set up a `config.py` per the comment below the import. It needs to have a variable called `league_team_info` that contains a list of tuples. The values in the tuple must be your nickname for the league, the league ID, and your team's ID within the league. 

In [17]:
import os
from espn_api.football import League
from config import league_team_info
#[('work-friends',1234567,7)]
from espn_ff_toolkit import get_espn_leagues 


swid = os.environ['ESPN_SWID']
espn_s2 = os.environ['ESPN_S2']

year=2024
league_dict = get_espn_leagues(league_team_info,year,espn_s2=espn_s2,swid=swid)

In [18]:
from espn_ff_toolkit import get_rosters, get_free_agents, get_roster_and_projections

def get_rosters_tool()-> str:
    "Get the rosters for all current leagues"
    return get_rosters(league_dict)

register_function(
    get_rosters_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the rosters for all current leagues"
)

def get_free_agents_tool() -> str:
    "Get the current free agents in all leagues"
    return get_free_agents(league_dict)

register_function(
    get_free_agents_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the current free agents in all leagues"
)

def get_roster_and_projections_tool() -> str:
    "Get the rosters for all current leagues and also all projected and past stats"
    return get_roster_and_projections(league_dict)

register_function(
    get_roster_and_projections_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the rosters for all current leagues and also all projected and past stats"
)

In [19]:
from datetime import datetime
def get_current_date() -> str:
    "Get the current date, because it is not October 2023"
    current_date = datetime.now()
    return str(current_date)

register_function(
    get_current_date,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Find out what the current date is",  # A description of the tool.
)

In [20]:
from draft_kings import Sport, Client

def get_draftkings_weekly_salaries():
    output = {}
    players_info = Client().available_players(draft_group_id=109136)
    for p in players_info.players:
        full_name = p.first_name + " " + p.last_name
        output[full_name] = p.draft_details.salary
    return output

dk_salaries = get_draftkings_weekly_salaries()

def check_draftkings_salary(player: str) -> str:
    "Check a player's salary on DraftKings. A higher salary means they are projected to do better"
    try:
        salary = dk_salaries[player]
        return(player, "salary on DraftKings is: ", salary)
    except:
        return(player, "has no salary on DraftKings this week")

register_function(
    check_draftkings_salary,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Check a player's salary on DraftKings. A higher salary means they are projected to do better",  # A description of the tool.
)


In [21]:
from autogen.agentchat.contrib.capabilities.text_compressors import LLMLingua
from autogen.agentchat.contrib.capabilities.transforms import TextMessageCompressor
from autogen.agentchat.contrib.capabilities import transform_messages

llm_lingua = LLMLingua()
text_compressor = TextMessageCompressor(text_compressor=llm_lingua)
context_handling = transform_messages.TransformMessages(transforms=[text_compressor])
context_handling.add_to_agent(assistant)

In [24]:
chat_result = user_proxy.initiate_chat(assistant, message="When will CMC be back?")

User (to Assistant):

When will CMC be back?

--------------------------------------------------------------------------------
4 tokens saved with text compression.

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_GP8WJOJoA6dGi92phUaZf9B8): get_current_date *****
Arguments: 
{}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_current_date...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_GP8WJOJoA6dGi92phUaZf9B8) *****
2024-10-01 21:10:33.942886
**********************************************************************

--------------------------------------------------------------------------------
9 tokens saved with text compression.

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_R0hU82qgoAor7tE2xmRGvpoQ): google_it *****
Arguments: 
{

c:\Users\alexl_g8yj9pc\anaconda3\envs\fantasy-llm\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(
c:\Users\alexl_g8yj9pc\anaconda3\envs\fantasy-llm\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



>>>>>>>> EXECUTING FUNCTION check_reddit...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_R0hU82qgoAor7tE2xmRGvpoQ) *****
Sep 6, 2024 ... Christian McCaffrey missed most of 49ers training camp with a strained calf, but he's progressing well ahead of San Francisco's 2024 NFL season opener. Sep 14, 2024 ... McCaffrey now will miss the next four games, at least, before the team assesses whether he can return in mid-October. ... status was. Reply ... Oct 20, 2023 ... CLEVELAND, OH - OCTOBER 15: San Francisco 49ers running back Christian McCaffrey (23. Oct 22, 2023 ... Sunday update — News continues to trend toward the positive for McCaffrey's ... October 23. Betting and DFS implications for Week 7. It's ... Mar 1, 2024 ... Photo by Chase Senior on October 01, 2024. May be an image of 2 ... 49er_edits. 49ers News, Updates, Edits & More .. Follow. jay.r400's ... Sep 14, 2024 ... How long is Christian McCaffrey out? Latest news, updates on 49ers RB 2024 s

In [25]:
from IPython.display import Markdown as md
md(chat_result.summary)

As of October 1, 2024, Christian McCaffrey is dealing with some injury issues. Here are the latest updates:

1. **Injury Status**: McCaffrey has been reported to have bilateral Achilles tendinitis, which is affecting both of his legs. It's a significant concern for his ongoing health and performance. 

2. **Recent News**: Reports indicate that although the San Francisco 49ers and McCaffrey himself aren't viewing his situation as season-ending, there is no clear timeline for his return. Fans and analysts are uncertain when he will be able to play again.

3. **Rehabilitation Progress**: There are indications that McCaffrey is back from a medical trip to Germany and he may begin running on hard ground soon. The goal appears to be for him to ramp up his rehab activities, potentially seeing some time back on the field by early November. 

In conclusion, while there is hope for his return, it's essential to monitor the situation closely as these updates develop. If you have him on your fantasy team, be prepared for possible continuing absences as he rehabilitates.

If you're considering adding or dropping McCaffrey from your fantasy roster, it might be wise to hold onto him a little longer if you have the roster flexibility, as he could be back in a couple of weeks. 

Would you like information on whether he is currently rostered or available as a free agent in your fantasy league? 

